In [1]:
import ee
import geopandas as gpd
import folium
import numpy as np
import rasterio
import matplotlib.pyplot as plt
from shapely.geometry import mapping


In [2]:
# Authenticate and Initialize Earth Engine
ee.Authenticate()
ee.Initialize()

In [3]:
#  Define AOI for Puerto Rico
aoi = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017").filter(ee.Filter.eq("country_na", "Puerto Rico"))

In [4]:
#  Define Pre and Post Event Dates for Hurricane Maria
pre_event_dates = ['2017-09-01', '2017-09-19']
post_event_dates = ['2017-09-20', '2017-10-10']

In [5]:
#  Load and Process Sentinel-1 SAR Data
def load_sar_data(date_range):
    return ee.ImageCollection('COPERNICUS/S1_GRD') \
        .filterBounds(aoi) \
        .filterDate(date_range[0], date_range[1]) \
        .filter(ee.Filter.eq('instrumentMode', 'IW')) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
        .select('VV') \
        .mean()

pre_event_sar = load_sar_data(pre_event_dates)
post_event_sar = load_sar_data(post_event_dates)

In [6]:
#  Speckle Filtering (Gamma Filter)
def apply_gamma_filter(image):
    return image.focal_mean(radius=50, kernelType='circle', iterations=1)

pre_event_filtered = apply_gamma_filter(pre_event_sar)
post_event_filtered = apply_gamma_filter(post_event_sar)

In [7]:
# Calculate Backscatter Change
backscatter_diff = post_event_filtered.subtract(pre_event_filtered)


In [8]:
# Load DEM and Slope Data
dem = ee.Image('USGS/SRTMGL1_003')
slope = ee.Terrain.slope(dem)

In [9]:
#  Define Thresholds
backscatter_threshold = -2.5
elevation_threshold = 50
slope_threshold = 5

In [10]:
# Create Flood Mask Using SAR, DEM, and Slope
flood_mask = backscatter_diff.lt(backscatter_threshold) \
    .And(dem.lt(elevation_threshold)) \
    .And(slope.lt(slope_threshold))


In [11]:
#  Remove Permanent Water Bodies (Use Updated Dataset)
permanent_water = ee.Image('JRC/GSW1_4/GlobalSurfaceWater').select('max_extent')
flood_mask_refined = flood_mask.updateMask(permanent_water.Not())

In [16]:
flood_vectors = flood_mask_refined.reduceToVectors(
    geometry=aoi.geometry(),  
    geometryType='polygon',
    reducer=ee.Reducer.countEvery(),
    scale=30,
    maxPixels=1e10
)

In [19]:
# Export to Google Drive
export_task = ee.batch.Export.table.toDrive(
    collection=flood_vectors,
    description='HurricaneMaria_FloodExtent',
    fileFormat='GeoJSON'
)
export_task.start()

In [25]:
task_status = ee.batch.Task.list()
for t in task_status:
    print(t.status())

{'state': 'COMPLETED', 'description': 'HurricaneMaria_FloodExtent', 'priority': 100, 'creation_timestamp_ms': 1744076491557, 'update_timestamp_ms': 1744076903426, 'start_timestamp_ms': 1744076496488, 'task_type': 'EXPORT_FEATURES', 'destination_uris': ['https://drive.google.com/'], 'attempt': 1, 'batch_eecu_usage_seconds': 22912.7265625, 'id': 'FSP7BU2LD3BGJ7O3RVZ54N3Z', 'name': 'projects/poststorm/operations/FSP7BU2LD3BGJ7O3RVZ54N3Z'}
{'state': 'RUNNING', 'description': 'HurricaneMaria_FloodExtent', 'priority': 100, 'creation_timestamp_ms': 1744076479682, 'update_timestamp_ms': 1744076953739, 'start_timestamp_ms': 1744076483636, 'task_type': 'EXPORT_FEATURES', 'attempt': 1, 'batch_eecu_usage_seconds': 7805.272460937, 'id': 'J7QV5JLJJBXKNNYYGCL3VD4U', 'name': 'projects/poststorm/operations/J7QV5JLJJBXKNNYYGCL3VD4U'}
{'state': 'FAILED', 'description': 'HurricaneMaria_FloodExtent', 'priority': 100, 'creation_timestamp_ms': 1744076278201, 'update_timestamp_ms': 1744076286932, 'start_times

In [30]:
# Local Processing (after download from Drive)
flood_gdf = gpd.read_file('C:\\Users\\hanis\\Downloads\\HurricaneMaria_FloodExtent.geojson')
municipalities_gdf = gpd.read_file('C:\\Users\\hanis\\Downloads\\pri_adm_2019_shp\\pri_admbnda_adm1_2019.shp')

In [31]:
# Ensure Same CRS
flood_gdf = flood_gdf.to_crs(municipalities_gdf.crs)

In [33]:
#  Spatial Join: Assign Flood to Municipality
flood_by_municipality = gpd.sjoin(flood_gdf, municipalities_gdf, how='inner', predicate='intersects')


In [34]:
#  Calculate Flood Area per Municipality
flood_by_municipality['flooded_area_km2'] = flood_by_municipality.geometry.area / 1e6
flood_by_municipality[['municipality_name', 'flooded_area_km2']].to_csv('FloodedAreaByMunicipality.csv', index=False)


C:\Users\hanis\AppData\Local\Temp\ipykernel_27016\773716852.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  flood_by_municipality['flooded_area_km2'] = flood_by_municipality.geometry.area / 1e6


KeyError: "['municipality_name'] not in index"

In [35]:
print(municipalities_gdf.columns)


Index(['ADM0_ES', 'ADM0_PCODE', 'ADM1_ES', 'ADM1_PCODE', 'geometry'], dtype='object')


In [36]:
municipalities_gdf = municipalities_gdf.rename(columns={'ADM1_ES': 'municipality_name'})


In [37]:
flood_by_municipality = gpd.sjoin(flood_gdf, municipalities_gdf, how='inner', predicate='intersects')


In [38]:
flood_by_municipality['flooded_area_km2'] = flood_by_municipality.geometry.area / 1e6
flood_by_municipality[['municipality_name', 'flooded_area_km2']].to_csv('FloodedAreaByMunicipality.csv', index=False)


C:\Users\hanis\AppData\Local\Temp\ipykernel_27016\2705238762.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  flood_by_municipality['flooded_area_km2'] = flood_by_municipality.geometry.area / 1e6


In [39]:
# Reproject to UTM Zone 20N
flood_by_municipality = flood_by_municipality.to_crs(epsg=32620)


In [40]:
# Calculate flooded area in square kilometers
flood_by_municipality['flooded_area_km2'] = flood_by_municipality.geometry.area / 1e6


In [41]:
# Export to CSV
flood_by_municipality[['municipality_name', 'flooded_area_km2']].to_csv('FloodedAreaByMunicipality.csv', index=False)


In [43]:
# Visualization Map (Folium)
m = folium.Map(location=[18.2208, -66.5901], zoom_start=8)

folium.GeoJson(
    municipalities_gdf,
    name='Municipalities',
    style_function=lambda feature: {
        'fillColor': 'none',
        'color': 'black',
        'weight': 1
    }
).add_to(m)

folium.GeoJson(
    flood_gdf,
    name='Flood Extent',
    style_function=lambda feature: {
        'fillColor': 'blue',
        'color': 'blue',
        'weight': 0.5,
        'fillOpacity': 0.6
    }
).add_to(m)

folium.LayerControl().add_to(m)
m.save("HurricaneMaria_Flood_Map.html")